In [ ]:
! pip install river

In [ ]:
import pandas as pd
import numpy as np
from river import compose
from river import feature_extraction
from river import linear_model
from river import metrics
from river import preprocessing
from river import ensemble
from river import optim
from river import facto

In [ ]:
places = pd.read_parquet('/content/drive/MyDrive/model_data/feature_df.parquet')

In [ ]:
pl = places[[
 'name', 'rating', 'combined_text', 'distance_to_Colombo',
 'distance_to_Kandy',
 'distance_to_Galle',
 'distance_to_Jaffna',
 'distance_to_Trincomalee',
 'distance_to_Sigiriya',
 'distance_to_Ella',
 'distance_to_Pollonnaruwa'
]]

In [ ]:
# Calculate shortest distance
distance_columns = ['distance_to_Colombo', 'distance_to_Kandy', 'distance_to_Galle',
                    'distance_to_Jaffna', 'distance_to_Trincomalee', 'distance_to_Sigiriya',
                    'distance_to_Ella', 'distance_to_Pollonnaruwa']
pl['shortest_distance_to'] = pl[distance_columns].min(axis=1)

<ipython-input-7-77869314d6d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl['shortest_distance_to'] = pl[distance_columns].min(axis=1)


In [ ]:
pl.head()

,name,rating,combined_text,distance_to_Colombo,distance_to_Kandy,distance_to_Galle,distance_to_Jaffna,distance_to_Trincomalee,distance_to_Sigiriya,distance_to_Ella,distance_to_Pollonnaruwa,shortest_distance_to
0,Arugam Bay Beach,4.800000,arugam bay beach surfer paradise spend incredi...,218.563424,141.941228,198.802269,370.357832,205.016806,171.427752,87.828900,155.518842,87.828900
1,Mirissa Beach,4.600000,mirissa beach truly gem sri lankas southern co...,127.194493,150.087678,28.982974,413.832059,303.951184,224.999764,121.530115,234.865936,28.982974
2,Weligama Beach (surf and stay),4.400000,weligama beach fantastic spot beginner experie...,123.222046,147.403949,25.403452,410.487250,301.725732,222.363738,120.366011,232.620786,25.403452
3,Ahangama,4.459437,ahangama bit disappointing solo traveler surfi...,119.083822,148.668171,17.932460,409.508778,303.985546,223.654664,124.958333,234.877202,17.932460
4,Hikkaduwa Beach,4.700000,hikkaduwa beach delightful escape solo travele...,91.173619,140.531383,16.399875,389.798174,297.621888,214.019935,132.511123,229.193326,16.399875


In [ ]:
pl_sent = pd.read_csv('/content/drive/MyDrive/model_data/Places Dataset Classified Reviews Cleaned Combined.csv')

In [ ]:
pl_sent.columns

Index(['name', 'lat', 'lng', 'formatted_address', 'rating',
       'user_ratings_total', 'latest_reviews', 'location_info', 'activities',
       'Geographical', 'Historical', 'Religious', 'Natural', 'Entertainment',
       'Accommodation', 'Shopping', 'Food', 'classified_reviews',
       'negetive_rate', 'combined_info'],
      dtype='object')

In [ ]:
# Merge the DataFrames based on the 'name' column
merged_df = pd.merge(pl, pl_sent[['name', 'negetive_rate']], on='name', how='left')

# Display the merged DataFrame
merged_df.head()

,name,rating,combined_text,distance_to_Colombo,distance_to_Kandy,distance_to_Galle,distance_to_Jaffna,distance_to_Trincomalee,distance_to_Sigiriya,distance_to_Ella,distance_to_Pollonnaruwa,shortest_distance_to,negetive_rate
0,Arugam Bay Beach,4.800000,arugam bay beach surfer paradise spend incredi...,218.563424,141.941228,198.802269,370.357832,205.016806,171.427752,87.828900,155.518842,87.828900,0.0
1,Mirissa Beach,4.600000,mirissa beach truly gem sri lankas southern co...,127.194493,150.087678,28.982974,413.832059,303.951184,224.999764,121.530115,234.865936,28.982974,0.0
2,Weligama Beach (surf and stay),4.400000,weligama beach fantastic spot beginner experie...,123.222046,147.403949,25.403452,410.487250,301.725732,222.363738,120.366011,232.620786,25.403452,0.0
3,Ahangama,4.459437,ahangama bit disappointing solo traveler surfi...,119.083822,148.668171,17.932460,409.508778,303.985546,223.654664,124.958333,234.877202,17.932460,0.2
4,Hikkaduwa Beach,4.700000,hikkaduwa beach delightful escape solo travele...,91.173619,140.531383,16.399875,389.798174,297.621888,214.019935,132.511123,229.193326,16.399875,0.0


In [ ]:
pl = merged_df.copy()

In [ ]:
user_info = pd.read_excel('/content/drive/MyDrive/model_data/Visitors Preference Dataset.xlsx')

In [ ]:
user_info.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle..."


In [ ]:
dataset = {}
for _, row in pl.iterrows():
    dataset[row['name']] = {
        'combined_text': row['combined_text'],
        'shortest_distance_to': row['shortest_distance_to'],
        'rating': row['rating'],
        'negetive_rate': row['negetive_rate']
    }

## tree based model

In [ ]:
from river import tree

In [ ]:
model = compose.Pipeline(
    ('tfidf', feature_extraction.TFIDF(lowercase=True, strip_accents=True, on='combined_text')),
    ('scale', preprocessing.StandardScaler()),
    ('tree', tree.HoeffdingAdaptiveTreeRegressor(
        max_depth=15,
        leaf_prediction='adaptive',
        grace_period=50,
        model_selector_decay=0.3,
        seed=0
    ))
)

In [ ]:
metric_mae = metrics.MAE()
metric_rmse = metrics.RMSE()
metric_r2 = metrics.R2()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

In [ ]:
def calculate_relevance(place_data, preferred_activities):
    score = 0
    for activity in preferred_activities:
        if activity.lower() in place_data['combined_text'].lower():
            score += 1
    return score / len(preferred_activities) if preferred_activities else 0

def calculate_similarity(place1, place2):
    text1 = dataset[place1]['combined_text']
    text2 = dataset[place2]['combined_text']

    tfidf = TfidfVectorizer().fit_transform([text1, text2])
    return 1 - cosine(tfidf.toarray()[0], tfidf.toarray()[1])

In [ ]:
def get_diverse_recommendations(user_id, places_dataset, user_info, top_n=5, diversity_threshold=0.3):
    user = user_info[user_info['User ID'] == user_id].iloc[0]
    bucket_list = user['Bucket list destinations Sri Lanka']

    all_recommendations = []

    for place, data in places_dataset.items():
        features = {
            'combined_text': data['combined_text'],
            'shortest_distance_to': data['shortest_distance_to']
        }

        # Predict the rating
        predicted_rating = model.predict_one(features)

        # Calculate bucket list score
        bucket_list_score = 1 if place in bucket_list else 0

        all_recommendations.append((place, predicted_rating, bucket_list_score, data['rating']))

    # Sort by predicted rating and bucket list score
    all_recommendations.sort(key=lambda x: (x[1] * 0.7 + x[2] * 0.3), reverse=True)

    diverse_recommendations = []
    for place, pred_rating, bucket_list_score, actual_rating in all_recommendations:
        if not diverse_recommendations or all(
            calculate_similarity(place, rec[0]) < diversity_threshold
            for rec in diverse_recommendations
        ):
            diverse_recommendations.append((place, pred_rating, bucket_list_score, actual_rating))

        if len(diverse_recommendations) == top_n:
            break

    return diverse_recommendations

In [ ]:
def train_model(dataset):
    for place, data in dataset.items():
        features = {
            'combined_text': data['combined_text'],
            'shortest_distance_to': data['shortest_distance_to']
        }
        rating = data['rating']

        # Make a prediction (for evaluation)
        pred = model.predict_one(features)

        # Update the metrics
        metric_mae.update(rating, pred)
        metric_rmse.update(rating, pred)
        metric_r2.update(rating, pred)

        # Train the model
        model.learn_one(features, rating)

In [ ]:
def get_recommendations(user_id, places_dataset, user_info, top_n=5):
    user = user_info[user_info['User ID'] == user_id].iloc[0]
    preferred_activities = user['Preferred Activities']
    bucket_list = user['Bucket list destinations Sri Lanka']

    recommendations = []

    for place, data in places_dataset.items():
        features = {
            'combined_text': data['combined_text'],
            'shortest_distance_to': data['shortest_distance_to']
        }

        # Predict the rating
        predicted_rating = model.predict_one(features)

        # Calculate relevance score
        relevance_score = calculate_relevance(data, preferred_activities)

        # Calculate bucket list score
        bucket_list_score = 1 if place in bucket_list else 0

        # Calculate diversity score
        diversity_score = calculate_diversity_score(recommendations, predicted_rating)

        # Combine scores
        combined_score = (predicted_rating * 0.3 + relevance_score * 0.3 +
                          bucket_list_score * 0.3 + diversity_score * 0.1)

        recommendations.append((place, combined_score, data['rating']))

    # Sort by combined score
    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:top_n]

In [ ]:
user_id = 1  # Jennifer Quinn's ID
recommendations = get_recommendations(user_id, dataset, user_info)

print(f"Top 5 Recommended Places for User {user_id}:")
for place, pred_rating, relevance, actual_rating in recommendations:
    print(f"{place}: Predicted Rating: {pred_rating:.2f}, Actual Rating: {actual_rating}, Relevance: {relevance:.2f}")

print("\nModel Performance Metrics:")
print(f"MAE: {metric_mae.get():.4f}")
print(f"RMSE: {metric_rmse.get():.4f}")
print(f"R2: {metric_r2.get():.4f}")

Top 5 Recommended Places for User 1:
Arugam Bay Beach: Predicted Rating: nan, Actual Rating: 4.8, Relevance: 0.82
Mirissa Beach: Predicted Rating: nan, Actual Rating: 4.6, Relevance: 0.82
Weligama Beach (surf and stay): Predicted Rating: nan, Actual Rating: 4.4, Relevance: 0.82
Ahangama: Predicted Rating: nan, Actual Rating: 4.45943661971831, Relevance: 0.82
Hikkaduwa Beach: Predicted Rating: nan, Actual Rating: 4.7, Relevance: 0.82

Model Performance Metrics:
MAE: nan
RMSE: nan
R2: nan


In [ ]:
# prompt: train for user_id 2 to 1000

for user_id in range(2, 1001):
  recommendations = get_recommendations(user_id, dataset, user_info)

  print(f"Top 5 Recommended Places for User {user_id}:")
  for place, pred_rating, relevance, actual_rating in recommendations:
      print(f"{place}: Predicted Rating: {pred_rating:.2f}, Actual Rating: {actual_rating}, Relevance: {relevance:.2f}")

print("\nModel Performance Metrics:")
print(f"MAE: {metric_mae.get():.4f}")
print(f"RMSE: {metric_rmse.get():.4f}")
print(f"R2: {metric_r2.get():.4f}")

Streaming output truncated to the last 5000 lines.
Diyaluma Falls: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.82
Laxapana Falls: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.82
Sunset-Island: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.82
Top 5 Recommended Places for User 169:
Yapahuwa Rock Fortress: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.83
Colombo City Tour: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.83
Diyaluma Falls: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.83
Laxapana Falls: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.83
Sunset-Island: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.83
Top 5 Recommended Places for User 170:
Yapahuwa Rock Fortress: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.79
Colombo City Tour: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.79
Diyaluma Falls: Predicted Rating: 4.48, Actual Rating: 5.0, Relevance: 0.79
Laxapana Falls:

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import ndcg_score

In [ ]:
# Compute Precision/Recall/F1 score
def compute_precision_recall_f1(true_ratings, predicted_ratings, threshold=3):
    true_labels = [1 if rating >= threshold else 0 for rating in true_ratings]
    predicted_labels = [1 if rating >= threshold else 0 for rating in predicted_ratings]
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='binary')
    return precision, recall, f1

true_ratings = [data['rating'] for data in dataset.values()]
predicted_ratings = [model.predict_one({
    'combined_text': data['combined_text'],
    'shortest_distance_to': data['shortest_distance_to']
}) for data in dataset.values()]

precision, recall, f1 = compute_precision_recall_f1(true_ratings, predicted_ratings)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.9899
Recall: 0.9949
F1 Score: 0.9924


In [ ]:
# Example NDCG calculation
def compute_ndcg(true_relevance, predicted_scores):
    return ndcg_score([true_relevance], [predicted_scores])

true_relevance = [data['rating'] for data in dataset.values()]
predicted_scores = [model.predict_one({
    'combined_text': data['combined_text'],
    'shortest_distance_to': data['shortest_distance_to']
}) for data in dataset.values()]

ndcg = compute_ndcg(true_relevance, predicted_scores)
print(f"NDCG: {ndcg:.4f}")

NDCG: 0.9836


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load place data
df_places = pl.copy()

# Prepare the data
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_places['combined_text'])

# Perform clustering
num_clusters = 10  # Adjust this based on your dataset
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_places['cluster'] = kmeans.fit_predict(tfidf_matrix)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get diverse recommendations
def get_diverse_recommendations(user_id, user_info, df_places, cosine_sim, top_n=5):
    user = user_info[user_info['User ID'] == user_id].iloc[0]
    preferred_activities = user['Preferred Activities']
    bucket_list = user['Bucket list destinations Sri Lanka']

    # Find places that match preferred activities
    matching_places = df_places[df_places['combined_text'].apply(lambda x: any(activity.lower() in x.lower() for activity in preferred_activities))]

    if matching_places.empty:
        matching_places = df_places  # If no matches, consider all places

    recommendations = []
    clusters_used = set()

    for _, place in matching_places.iterrows():
        if len(recommendations) >= top_n:
            break

        if place['cluster'] not in clusters_used:
            # Get similar places within the same cluster
            cluster_places = df_places[df_places['cluster'] == place['cluster']]
            place_index = df_places.index[df_places['name'] == place['name']].tolist()[0]

            # Calculate similarity scores
            sim_scores = list(enumerate(cosine_sim[place_index]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = [i for i in sim_scores if i[0] in cluster_places.index]

            # Get the most similar place in the cluster
            if sim_scores:
                top_similar = df_places.iloc[sim_scores[0][0]]

                # Calculate a score based on similarity, rating, and bucket list
                similarity_score = sim_scores[0][1]
                rating_score = top_similar['rating'] / 5.0  # Normalize rating to 0-1 range
                bucket_list_score = 1 if top_similar['name'] in bucket_list else 0

                total_score = (similarity_score * 0.4 + rating_score * 0.4 + bucket_list_score * 0.2)

                recommendations.append((top_similar['name'], total_score))
                clusters_used.add(place['cluster'])

    # If we don't have enough recommendations, add from unused clusters
    while len(recommendations) < top_n:
        unused_clusters = set(range(num_clusters)) - clusters_used
        if not unused_clusters:
            break

        cluster = np.random.choice(list(unused_clusters))
        cluster_places = df_places[df_places['cluster'] == cluster]
        if not cluster_places.empty:
            top_place = cluster_places.sort_values('rating', ascending=False).iloc[0]
            recommendations.append((top_place['name'], top_place['rating'] / 5.0))
            clusters_used.add(cluster)

    # Sort recommendations by score
    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:top_n]

# Example usage
user_id = 1  # Replace with actual user ID
recommendations = get_diverse_recommendations(user_id, user_info, df_places, cosine_sim)

print(f"Top {len(recommendations)} Diverse Recommended Places for User {user_id}:")
for place, score in recommendations:
    print(f"{place}: Score: {score:.2f}")

# Function to evaluate diversity
def evaluate_diversity(recommendations, df_places):
    recommended_places = [place for place, _ in recommendations]
    clusters = df_places[df_places['name'].isin(recommended_places)]['cluster']
    unique_clusters = clusters.nunique()
    cluster_diversity = unique_clusters / len(recommendations)

    print(f"Cluster Diversity: {cluster_diversity:.2f}")
    print(f"Unique Clusters: {unique_clusters} out of {len(recommendations)} recommendations")

# Evaluate diversity
evaluate_diversity(recommendations, df_places)

Top 5 Diverse Recommended Places for User 1:
Arugam Bay Beach: Score: 0.78
Galle Dutch Fort: Score: 0.77
Sinharaja Forest Reserve: Score: 0.74
Polonnaruwa Ancient City: Score: 0.74
Yala National Park: Score: 0.73
Cluster Diversity: 1.00
Unique Clusters: 5 out of 5 recommendations
